<a href="https://colab.research.google.com/github/nhareesha/MLAI/blob/main/LLM/finetuned/Attempt_5_sql_data_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Need A100 processing units to run.

In [55]:
pip install pandas datasets


In [56]:
import os
import torch
import random
import numpy as np
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

# Step 0: Set Seed for Reproducibility

In [57]:
# Step 0: Set Seed for Reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(402)  # Set your seed here

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Step 1: Load the Spider Dataset


In [59]:
def load_spider_data(split_type='train'):
    # Load the Spider dataset
    dataset = load_dataset('spider', split=split_type)  # Use the split you need (train, validation, test)
    return dataset

In [31]:
ds = load_spider_data()
type(ds)
print(ds[:2])

{'db_id': ['department_management', 'department_management'], 'query': ['SELECT count(*) FROM head WHERE age  >  56', 'SELECT name ,  born_state ,  age FROM head ORDER BY age'], 'question': ['How many heads of the departments are older than 56 ?', 'List the name, born state and age of the heads of departments ordered by age.'], 'query_toks': [['SELECT', 'count', '(', '*', ')', 'FROM', 'head', 'WHERE', 'age', '>', '56'], ['SELECT', 'name', ',', 'born_state', ',', 'age', 'FROM', 'head', 'ORDER', 'BY', 'age']], 'query_toks_no_value': [['select', 'count', '(', '*', ')', 'from', 'head', 'where', 'age', '>', 'value'], ['select', 'name', ',', 'born_state', ',', 'age', 'from', 'head', 'order', 'by', 'age']], 'question_toks': [['How', 'many', 'heads', 'of', 'the', 'departments', 'are', 'older', 'than', '56', '?'], ['List', 'the', 'name', ',', 'born', 'state', 'and', 'age', 'of', 'the', 'heads', 'of', 'departments', 'ordered', 'by', 'age', '.']]}


# Step 2: Prepare the SQLDataset class


In [60]:
class SQLDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question = self.data[idx]['question']  # Get question
        answer = self.data[idx]['query']        # Get corresponding SQL from 'query' key

        # Tokenize the inputs and labels
        inputs = self.tokenizer(
            question, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt"
        )

        # You can choose which tokens to use for labels:
        # Here we can use both the actual SQL query and the tokenized versions
        labels = self.tokenizer(
            answer, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt"
        )

        # Tokenize `query_toks` and `query_toks_no_value` if needed
        query_toks = self.data[idx]['query_toks']
        query_toks_no_value = self.data[idx]['query_toks_no_value']

        # You might want to process query_toks and query_toks_no_value for additional insights or features

        # Flatten the tensor dimensions
        input_ids = inputs.input_ids.squeeze()
        attention_mask = inputs.attention_mask.squeeze()
        labels = labels.input_ids.squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
            'query_toks': query_toks,  # Return raw tokenized query
            'query_toks_no_value': query_toks_no_value  # Return raw tokenized query without values
        }



# Collate function to make all fileds of same length

pad the sequences in the batch to ensure they all have the same length. This is typically done with a custom collate function in the DataLoader

In [61]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # Extracting the relevant fields
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels = [item['labels'] for item in batch]

    # Pad the sequences to the maximum length in the batch
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask_padded = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=-100)  # Use -100 to ignore padding in the loss function

    return {
        'input_ids': input_ids_padded,
        'attention_mask': attention_mask_padded,
        'labels': labels_padded
    }


# Step 3: Load Pre-Trained Model and Tokenizer


In [ ]:
# Original base model - Can be used for ZERO SHOT referencing

model_name = 't5-small'  # or the name of your trained model
tokenizer = T5Tokenizer.from_pretrained(model_name)
original_model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [34]:
# Load already finetuned model

model_path = '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5/finetuned_model'
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [62]:
# Model for fine tuning
model_name = 't5-small'  # or the name of your trained model
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Load previous checkpoint if exists

In [35]:
# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [63]:
# Create output directory and checkpoint directory if they don't exist
output_dir = '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5/run3'
checkpoint_dir = os.path.join(output_dir, 'checkpoint')
os.makedirs(checkpoint_dir, exist_ok=True)


#IGNORE for sometime

In [64]:

# Define your checkpoint path
previous_checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5/checkpoint/checkpoint-0.pt'


In [65]:
if os.path.exists(previous_checkpoint_path):
    checkpoint = torch.load(previous_checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
else:
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
    start_epoch = 0


# Step 4: Load and Prepare Data



In [66]:
from torch.utils.data import ConcatDataset

# Load the Spider Dataset
spider_dataset_everything = load_spider_data()
spider_dataset_train = load_spider_data('train')
spider_dataset_val = load_spider_data('validation')

# Combine the datasets
spider_dataset_combined = ConcatDataset([spider_dataset_train, spider_dataset_val])
len(spider_dataset_combined)

8034

## Can be used for continuous learning

In [ ]:
#start_index = 1
#end_index = 1000

# Set the starting index
start_index = 1001

# Get the end index, which is the length of the dataset
end_index = len(spider_dataset_everything)

In [ ]:
print(len(spider_dataset_train))

In [67]:

# Select based in indexes - Optinal#
# spider_dataset = spider_dataset_everything.select(range(start_index, end_index))

spider_dataset = spider_dataset_combined

# Define the sizes for the splits
train_size = int(0.8 * len(spider_dataset))
val_size = int(0.1 * len(spider_dataset))  # 10% for validation
test_size = len(spider_dataset) - train_size - val_size  # Remaining 10% for testing

# Split the dataset into train, validation, and test sets
train_dataset, temp_dataset = torch.utils.data.random_split(spider_dataset, [train_size, len(spider_dataset) - train_size])
val_dataset, test_dataset = torch.utils.data.random_split(temp_dataset, [val_size, test_size])



In [68]:
# Now, train_dataset, val_dataset, and test_dataset are ready to use
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

Training set size: 6427
Validation set size: 803
Test set size: 804


# Create DataLoaders

## dataset with collate function

In [69]:
# Create DataLoaders with the custom collate function
train_data_loader = DataLoader(SQLDataset(train_dataset, tokenizer), batch_size=8, shuffle=True, collate_fn=collate_fn)
val_data_loader = DataLoader(SQLDataset(val_dataset, tokenizer), batch_size=8, collate_fn=collate_fn)
test_data_loader = DataLoader(SQLDataset(test_dataset, tokenizer), batch_size=8, collate_fn=collate_fn)

# Step 5: Setup Training Arguments

In [70]:
training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy='epoch',
    learning_rate=5e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    save_total_limit=2,  # Limit the total amount of checkpoints, delete the older checkpoints.
    save_steps=100,      # Save checkpoint every 500 steps

)

# Step 6: Initialize Trainer

In [71]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data_loader.dataset,
    eval_dataset=val_data_loader.dataset,
)

# Step 7: Train the Model

#### Train the Model with Checkpointing


In [72]:
for epoch in range(start_epoch, training_args.num_train_epochs):
    trainer.train()

    # Save the model checkpoint
    checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint-{epoch + 1}.pt')  # Specify the file name

    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_path)


Epoch,Training Loss,Validation Loss
1,0.193600,0.047757
2,0.051100,0.035705
3,0.043000,0.032397


Epoch,Training Loss,Validation Loss
1,0.041000,0.028097
2,0.028800,0.023825
3,0.024900,0.022499


Epoch,Training Loss,Validation Loss
1,0.025700,0.021579
2,0.019500,0.019251
3,0.016700,0.018583


# Step 8: Evaluate the Model


In [73]:
# Step 8: Evaluate the Model
results = trainer.evaluate()

In [74]:
# Print evaluation results
print(f"Evaluation results: {results}")

Evaluation results: {'eval_loss': 0.018582770600914955, 'eval_runtime': 6.205, 'eval_samples_per_second': 129.412, 'eval_steps_per_second': 16.277, 'epoch': 3.0}


# Step 9: Generate Predictions on Test Set -------- NEED TO FIX

In [86]:
predictions = []
actuals = []

for batch in test_data_loader:
    input_ids = batch['input_ids'].to(model.device)
    attention_mask = batch['attention_mask'].to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=1024,          # Increase max length
            num_beams=5,              # Use beam search
            early_stopping=True,      # Stop when output is complete
            temperature=1.0,          # Control randomness
            top_k=50,                 # Limit choices
            top_p=0.95                # Nucleus sampling
        )

    predicted_sql = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    actual_sql = tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)

    predictions.extend(predicted_sql)
    actuals.extend(actual_sql)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [75]:

predictions = []
actuals = []

for batch in test_data_loader:
    input_ids = batch['input_ids'].to(model.device)
    attention_mask = batch['attention_mask'].to(model.device)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids,
                                 attention_mask=attention_mask)

    predicted_sql = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    actual_sql = tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)

    predictions.extend(predicted_sql)
    actuals.extend(actual_sql)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


# Compare predicted SQL with actual SQL

In [87]:
# Compare predicted SQL with actual SQL
for pred, actual in zip(predictions, actuals):
    print(f"Predicted SQL: {pred}\nActual SQL: {actual}\n")

Predicted SQL: SELECT T1.name, T1.team_id FROM postseason AS T1 JOIN team AS T2 ON T1.team_id = T2.team_id_winner WHERE T2.year = 2008 ORDER BY T2.wins DESC LIMIT 1
Actual SQL: SELECT T2.name, T1.team_id_winner FROM postseason AS T1 JOIN team AS T2 ON T1.team_id_winner = T2.team_id_br WHERE T1.year = 2008 GROUP BY T1.team_id_winner ORDER BY count(*) DESC LIMIT 1;

Predicted SQL: SELECT Reign, Days_held FROM wrestler
Actual SQL: SELECT Reign, Days_held FROM wrestler

Predicted SQL: SELECT allergytype, count(*) FROM Has_allergy GROUP BY allergytype
Actual SQL: SELECT T2.allergytype, count(*) FROM Has_allergy AS T1 JOIN Allergy_type AS T2 ON T1.allergy = T2.allergy GROUP BY T2.allergytype

Predicted SQL: SELECT count(*) FROM STUDENT AS T1 JOIN VOTING_RECORD AS T2 ON T1.StuID = Class_Senator_Vote WHERE T1.sex = "M" AND T2.Election_Cycle = "Fall"
Actual SQL: SELECT count(*) FROM STUDENT AS T1 JOIN VOTING_RECORD AS T2 ON T1.StuID = Class_Senator_Vote WHERE T1.Sex = "M" AND T2.Election_Cycle 

# Step 10: Save the Fine-Tuned Model and Tokenizer

In [88]:
# Define the output directory
output_dir = '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5/finetuned_model/run2'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Save the fine-tuned model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5/finetuned_model/run2/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5/finetuned_model/run2/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5/finetuned_model/run2/spiece.model',
 '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/attempt_5/finetuned_model/run2/added_tokens.json')

# Step 11: Using the Model for Prompting

In [89]:
def generate_sql(question, max_length):
    input_ids = tokenizer.encode(question, return_tensors='pt').to(model.device)
    with torch.no_grad():
        output = model.generate(input_ids, max_new_tokens=max_length)
    generated_sql = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_sql


In [90]:
# Example prompt usage
question = "How many heads of the departments are older than 56?"
predicted_sql_query = generate_sql(question, 5000)
print(f"Generated SQL Query: {predicted_sql_query}")

Generated SQL Query: SELECT count(*) FROM head WHERE age > 56


# Smaller dataset results in underfitting - HENCE model is not able to learn much.

Things to do

1. Increase training set
2. Adjust learning rate
3. Create tailored prompts.

In [91]:
question='For those employees who did not have any job in the past, give me the comparison about the amount of job_id over the job_id , and group by attribute job_id, and list from low to high by the JOB_ID please.'

predicted_sql_query = generate_sql(question, 200)
print(f"Generated SQL Query: {predicted_sql_query}")

Generated SQL Query: select count(*), group_by_job_id from employees where job_id not in ( select job_id from employees where job_history ), job_id from employees where job_history ), job_id from employees where job_history!= 'yes' )


In [92]:
question='What total employees in Employee table?'

predicted_sql_query = generate_sql(question, 2000)
print(f"Generated SQL Query: {predicted_sql_query}")

Generated SQL Query: SELECT sum(employee_id) FROM Employees


In [93]:
question='There is a department table with columns department_id employee_id department_name.Employee table with employee_id and department_name.Join department and employee tables based on employee_id column?'

predicted_sql_query = generate_sql(question, 2000)
print(f"Generated SQL Query: {predicted_sql_query}")

Generated SQL Query: SELECT department_id, department_name FROM employees UNION SELECT department_id, department_name FROM employees WHERE department_id = 'Employee'
